In [1]:
import plotly.express as px
import plotly
plotly.io.templates.default = 'plotly' #'plotly_dark'
import pandas as pd
import sqlalchemy as sal

In [2]:
import pickle

In [3]:
with open("traffic_pred_horizon12.pkl", "rb") as fid:
    pred=pickle.load(fid)

In [4]:
#pred[1]["400001"]["time"]
pred[12]["400001"].head()
#pred[12].keys()

,pred,truth,time
0,71.122833,69.300003,2020-01-01 01:05:00
1,59.936764,50.700001,2020-01-01 01:10:00
2,43.150902,55.500000,2020-01-01 01:15:00
3,70.678955,70.699997,2020-01-01 01:20:00
4,67.980453,66.300003,2020-01-01 01:25:00


In [5]:
pred[12]["400001"].head()

,pred,truth,time
0,71.122833,69.300003,2020-01-01 01:05:00
1,59.936764,50.700001,2020-01-01 01:10:00
2,43.150902,55.500000,2020-01-01 01:15:00
3,70.678955,70.699997,2020-01-01 01:20:00
4,67.980453,66.300003,2020-01-01 01:25:00


In [6]:
import numpy as np
def predictions_to_df(horizon=12, timestamp="2020-06-29 22:45:00"):
    odf=pd.DataFrame(columns=["pred_speed", "pred_error"])
    
    sid_w_time=list(pred[horizon].keys())[0]
    
    tidx = np.where(pred[horizon][sid_w_time]["time"] == timestamp)[0][0]
    print(tidx)
    
    for sid in pred[horizon].keys():
        isid=int(sid)
        #print(sid)
        try:
            odf.loc[isid,"pred_speed"] = float(pred[horizon][sid].loc[tidx,"pred"])
            odf.loc[isid,"pred_error"] = float(pred[horizon][sid].loc[tidx,"pred"] - pred[horizon][sid].loc[tidx,"truth"])
        except AttributeError:
            pass
    
    return odf

odf=predictions_to_df()

52085


In [7]:
len(odf)

320

In [8]:
host="capstone.clihskgj8i7s.us-west-2.rds.amazonaws.com"
username="group3"
db="db1"
#pw=getpass.getpass("Enter database password")
pw=open("/home/daniel/Desktop/.awsdb","r").read().rstrip()

In [9]:
engine = sal.create_engine('postgresql://%s:%s@%s/%s' % (username, pw, host, db))

In [10]:
#download a snapshot of data
query="""
select l.sid, l.longitude, l.latitude, l.stype, l.fwy ,l.direc, l.abs_pm, l.state_pm
from pemslocs l
"""
sensors=pd.read_sql(query, engine, index_col="sid")
#sensors.head()

In [11]:
snap=pd.merge(sensors, odf, left_index=True, right_index=True)
snap.head()

,longitude,latitude,stype,fwy,direc,abs_pm,state_pm,pred_speed,pred_error
400001,-121.901149,37.364085,ML,101,N,387.897,38.26,71.1937,-0.106308
400017,-121.945440,37.253303,ML,85,S,10.070,10.07,62.151,0.450962
400030,-121.906538,37.359087,ML,880,S,3.640,3.64,67.6505,-0.649521
400040,-121.873109,37.294949,ML,87,S,2.860,2.87,67.2779,-0.122131
400045,-121.902233,37.363402,ML,880,N,4.020,4.02,62.0328,-0.167202


In [12]:
#pd.read_sql("select * from pemslocs limit 1", engine)

In [13]:
#convert from object to float
for col in ["pred_speed", "pred_error"]:
    snap[col]=snap[col].astype(float)

In [14]:
#create new column with fwy and direction as string
snap["fwdir"] = snap.apply(lambda x: str(x["fwy"]) + str(x["direc"]), axis=1)
snap["fwdir"].unique()

array(['101N', '85S', '880S', '87S', '880N', '85N', '101S', '17S', '280S',
       '237W', '237E', '87N', '17N', '280N', '680S', '680N'], dtype=object)

In [15]:
#function to move fwys in separate directions apart for visualization
def spread_lanes(sdf, fwdir, latspread=0.0005, lonspread=0.0005):
    direc=fwdir[-1]
    
    # this assumes right-hand traffic
    if direc == 'W':
         sdf["latitude"] += latspread
    elif direc == "E":
        sdf["latitude"] -= latspread
    elif direc == "S":
        sdf["longitude"] -= lonspread
    elif direc == "N":
        sdf["longitude"] += lonspread
    
    return sdf
    
#selhr=pd.read_sql("select * from usdot u where u.fwdir = '94W' order by u.abs_pm", engine)
#display(selhr.head(n=1))
#spread_lanes(selhr, '94W').head(n=1)

In [26]:
hrdfs=[]

from scipy.interpolate import interp1d

def interpolate_onto_hr_roads(indf, cols=["avg_speed"], dropna = False, do_spread_lanes=True):

    for fwdir in indf["fwdir"].unique():
        print("\rInterpolating average speed along %s " % fwdir, end="")
        sel=indf.query('fwdir == "%s"' % fwdir).sort_values(by="abs_pm")
        selhr=pd.read_sql("select * from usdot u where u.fwdir = '%s' order by u.abs_pm" % fwdir, engine)
        
        if do_spread_lanes:
            selhr=spread_lanes(selhr, fwdir)

        if len(sel) > 1:
            #fillvalmin=sel.loc[minidx,"abs_pm"]
            #fillvalmax=sel.loc[maxidx,"abs_pm"]
            for col in cols:
                interpmod = interp1d(sel["abs_pm"].values, sel[col].values, bounds_error=False)
                selhr[col] = interpmod(selhr["abs_pm"].values)
                if dropna:
                    selhr.dropna(axis="index", how="any", subset=[col], inplace=True)
            hrdfs.append(selhr)
    print(" done.")

    selhr=pd.concat(hrdfs)
    selhr.reset_index(drop=True, inplace=True)
    
    return selhr

selhr=interpolate_onto_hr_roads(snap, cols=["pred_speed", "pred_error"])

Interpolating average speed along 680N  done.


In [30]:
def speed2cat(speed):
    if np.isnan(speed):
        return 'no data'
    elif speed > 65:
        return 'fast'
    elif speed > 60:
        return 'moderate'
    elif speed > 50:
        return 'slow'
    else: 
        return 'very slow'
        
selhr["speed_cat"]=selhr["pred_speed"].apply(speed2cat)

In [31]:
selhr["pred_error"].describe()

count    4300.000000
mean        0.121583
std         1.290462
min        -9.883328
25%        -0.369785
50%         0.027274
75%         0.570750
max         6.580444
Name: pred_error, dtype: float64

In [34]:
def err2cat(err):
    if np.isnan(err):
        return 'no data'
    elif err > 5:
        return 'overpredict > 5 mph'
    elif err > 1:
        return 'overpredict 1 - 5 mph'
    elif err > -1: 
        return 'within 1 mph'
    elif err > -5:
        return 'underpredict 1 - 5 mph'
    else:
        return 'underpredict > 5 mph'
        
selhr["err_cat"]=selhr["pred_error"].apply(err2cat)

In [35]:
selhr.tail()

,usdid,fwdir,latitude,longitude,abs_pm,pred_speed,pred_error,speed_cat,err_cat
7905,5725,680N,37.463662,-121.902761,9.858517,NaN,NaN,no data,no data
7906,5726,680N,37.463810,-121.902878,9.870589,NaN,NaN,no data,no data
7907,5727,680N,37.463950,-121.902988,9.881989,NaN,NaN,no data,no data
7908,5728,680N,37.464358,-121.903274,9.914249,NaN,NaN,no data,no data
7909,5729,680N,37.464540,-121.903417,9.929069,NaN,NaN,no data,no data


In [38]:
# %load road_group_split.py
def road_group_split(indf, splitcat="speed_cat", 
                     keep_cols=["latitude", "longitude", "abs_pm", "pred_speed","fwdir"]):
    
    odict={}
    for col in keep_cols + [splitcat]:
        odict[col] = []
    #lats=[]
    #lons=[]
    #abs_pm=[]
    #cat=[]
    odict["linegroup"] = []
    
    p=0 #number holding line group
    i0=indf.index[0]
    
    for col in keep_cols + [splitcat]:
        odict[col].append(indf.loc[i0,col])
    
    #lats.append(indf.loc[i0, "latitude"])
    #lons.append(indf.loc[i0, "longitude"])
    #abs_pm.append(indf.loc[i0, "abs_pm"])
    #cat.append(indf.loc[i0, splitcat])
    #group.append(p)
    odict["linegroup"].append(p)
    
    for n in indf.index[1:]:
        if indf.loc[n,"fwdir"] != indf.loc[i0,"fwdir"]:
            p+=1
            
        if indf.loc[n,splitcat] != indf.loc[i0,splitcat] and indf.loc[n,"fwdir"] == indf.loc[i0,"fwdir"]:
            for col in keep_cols:
                odict[col].append(indf.loc[n,col])
            odict[splitcat].append(indf.loc[i0,splitcat])
            #end previous group with new point and speed cat from previous point
            #lats.append(indf.loc[n,"latitude"])
            #lons.append(indf.loc[n,"longitude"])
            #lons.append(indf.loc[n,"abs_pm"])
            #cat.append(indf.loc[i0,splitcat])
            #group.append(p)
            odict["linegroup"].append(p)
            p+=1
        #start new group
        for col in keep_cols + [splitcat]:
            odict[col].append(indf.loc[n,col])
        #lats.append(indf.loc[n,"latitude"])
        #lons.append(indf.loc[n,"longitude"])
        #cat.append(indf.loc[n,splitcat])
        #group.append(p)        
        odict["linegroup"].append(p)
        i0=n
    
    #odf=pd.DataFrame({"latitude": lats, "longitude": lons, splitcat: cat, "linegroup": group})
    odf=pd.DataFrame(odict)
    return odf

selhrg=road_group_split(selhr)
selhrg.tail()

,latitude,longitude,abs_pm,pred_speed,fwdir,speed_cat,linegroup
8052,37.463662,-121.902761,9.858517,NaN,680N,no data,162
8053,37.463810,-121.902878,9.870589,NaN,680N,no data,162
8054,37.463950,-121.902988,9.881989,NaN,680N,no data,162
8055,37.464358,-121.903274,9.914249,NaN,680N,no data,162
8056,37.464540,-121.903417,9.929069,NaN,680N,no data,162


In [20]:
cdm={"fast": "green",
     "moderate": "orange",
     "slow": "red",
     "very slow": "brown",
     "no data": "gray"}

In [23]:
plotly.io.templates.default='plotly_dark'
#del fig
fig = px.line_mapbox(
#fig=px.scatter_mapbox
    selhrg, lat="latitude", lon="longitude", color="speed_cat",
                        #mapbox_style="carto-darkmatter",
                        mapbox_style="carto-positron",
                        #color_continuous_scale="aggrnyl" ,
                        line_group = "linegroup",
                        hover_data=["abs_pm", "pred_speed"],
                        center = {"lat": 37.34, "lon": -121.93},
                        color_discrete_map= cdm,
                        zoom=11,
                        height=800, width=1200)

for dat in fig.data:
    dat["line"]["width"] = 5

snap["size"] = 3
fig2 = px.scatter_mapbox(snap, lat="latitude", lon="longitude",
                        hover_data=["stype","fwy","direc","abs_pm","pred_speed"],
                        mapbox_style="stamen-terrain",
                         color="stype",
                        color_discrete_map={"ML": "blue"},
                        size="size",
                        opacity=0.35,
                        size_max=5)
#for dat in fig2.data:
fig.add_trace(fig2.data[0])
fig
#px.scatter(sel, x="longitude", y="latitude", color="total_flow").update_traces(mode='lines+markers')

In [ ]:
fig.write_image("traffic_layer_bay_w_usdot.jpg")

In [45]:
selhrg=road_group_split(selhr, splitcat="err_cat", 
                        keep_cols=["latitude", "longitude", "fwdir","abs_pm", "pred_speed", "pred_error"])

In [46]:
selhrg["err_cat"].unique()

array(['no data', 'within 1 mph', 'underpredict 1 - 5 mph',
       'overpredict 1 - 5 mph', 'overpredict > 5 mph',
       'underpredict > 5 mph'], dtype=object)

In [47]:
cdme={"within 1 mph": "green",
     "underpredict 1 - 5 mph": "orange",
     "underpredict > 5 mph": "red",
     "overpredict 1 - 5 mph": "lightgreen",
     "overpredict > 5 mph": "darkgreen",
     "no data": "gray"}

In [49]:
plotly.io.templates.default='plotly_dark'
#del fig
fig = px.line_mapbox(
#fig=px.scatter_mapbox
    selhrg, lat="latitude", lon="longitude", color="err_cat",
                        #mapbox_style="carto-darkmatter",
                        mapbox_style="carto-positron",
                        #color_continuous_scale="aggrnyl" ,
                        line_group = "linegroup",
                        hover_data=["abs_pm", "pred_speed"],
                        center = {"lat": 37.34, "lon": -121.93},
                        color_discrete_map= cdme,
                        zoom=10,
                        height=800, width=1200)

for dat in fig.data:
    dat["line"]["width"] = 5

snap["size"] = 3
fig2 = px.scatter_mapbox(snap, lat="latitude", lon="longitude",
                        hover_data=["stype","fwy","direc","abs_pm","pred_speed"],
                        mapbox_style="stamen-terrain",
                         color="stype",
                        color_discrete_map={"ML": "blue"},
                        size="size",
                        opacity=0.35,
                        size_max=5)
#for dat in fig2.data:
fig.add_trace(fig2.data[0])
fig
#px.scatter(sel, x="longitude", y="latitude", color="total_flow").update_traces(mode='lines+markers')